In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
import missingno as msno
import matplotlib.pyplot as plt
#I had the same issue.

#There seems to be an issue with the source from which Plotly is loaded. Changing the notebook mode might help.

from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from lightgbm import LGBMClassifier
import lightgbm as lgb
#import lazypredict
#from lazypredict.Supervised import LazyClassifier

import time
import warnings
warnings.filterwarnings('ignore')

from typing import List,Union,Any,Callable,Dict

from tqdm import tqdm
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

SEED = 42
fix_seed(SEED)

# Using Pandas to get familier with Your Data

we'll look at data recovered from the spaceship’s damaged computer system.
The train data is at the file path /kaggle/input/spaceship-titanic/train.csv.
We load and explore the data with the following commands:

In [ ]:
org_train = pd.read_csv(filepath_or_buffer='/kaggle/input/spaceship-titanic/train.csv')
org_train

# Interpreting Data Description
The results show 8 numbers for each column in your original dataset. The first number, the **count**,  shows how many rows have non-missing values.  

The second value is the **mean**, which is the average.  Under that, **std** is the standard deviation, which measures how numerically spread out the values are.

To interpret the **min**, **25%**, **50%**, **75%** and **max** values, imagine sorting each column from lowest to highest value.  The first (smallest) value is the min.  If you go a quarter way through the list, you'll find a number that is bigger than 25% of the values and smaller than 75% of the values.  That is the **25%** value (pronounced "25th percentile").  The 50th and 75th percentiles are defined analogously, and the **max** is the largest number.

In [ ]:
org_train.describe()

In [ ]:
org_train.isna().sum()

In [ ]:
org_train.nunique()

In [ ]:
org_test = pd.read_csv(filepath_or_buffer='/kaggle/input/spaceship-titanic/test.csv') # test data 
org_test

In [ ]:
org_test.describe()

In [ ]:
df = pd.concat([org_train,org_test],axis=0)

In [ ]:
df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand=True)

In [ ]:
def get_split_column(df, char: str, target_col_name: str, new_columns: List[str]):

    df[new_columns] = df[target_col_name].str.split(char, expand=True)
    
    return df

In [ ]:
get_split_column(df = df, char = '/', target_col_name = 'Cabin', new_columns = ['a', 'b', 'c'])

In [ ]:
train = df[:len(org_train)]

In [ ]:
test = df[len(org_train):]
test

In [ ]:
submission = pd.read_csv(filepath_or_buffer='/kaggle/input/spaceship-titanic/sample_submission.csv') # test data 
submission

In [ ]:
pd.DataFrame(train.isna().sum())

In [ ]:
train.nunique()

In [ ]:
def get_dataframe_info(df):
    """
    input
       df -> DataFrame
    output
       df_all_num_non_null -> DataFrame Info (sorted)
    """

    df_types = pd.DataFrame(df.dtypes)
    df_nulls = df.count()
    df_num_na = pd.DataFrame(df.isna().sum())
    df_num_unique = pd.DataFrame(df.nunique())
    
    df_null_count = pd.concat([df_types, df_nulls, df_num_na, df_num_unique], axis=1)
    df_null_count = df_null_count.reset_index()
    

    
    # Reassign column names
    col_names = ['features', 'types', 'num_non_null', 'num_null', 'num_unique']
    df_null_count.columns = col_names
    
    # Add this to sort
    df_all_num_non_null = df_null_count.sort_values(by=["types"], ascending=False)
    
    return df_all_num_non_null

get_dataframe_info(train)

In [ ]:
get_dataframe_info(test)

In [ ]:
pd.DataFrame(data=train.columns).T

In [ ]:
pd.DataFrame(data = train.isna().sum().sort_values(ascending= False), columns=['num_missing_data'])

In [ ]:
pd.DataFrame(data = test.nunique(), columns=['num_uniqe_data'])

# Label encodig
we change every categorical data to a number.That is each type will be subtuted by a number.for example we will substitute 1 for Grandmaster,2 for master ,3 for expert etc.. For implementing this we will first import Labelencoder from sklearn module.

Before getting into encoding,I will just breif you with types data variables present in this data:

Binary data : A binary variable a variable that has only 2 values..ie 0/1
Categorical data : A categorical variable is a variable that can take some limited number of values.for example,day of the week.It can be one of 1,2,3,4,5,6,7 only.
Ordinal data : An ordinal variable is a categorical variable that has some order associated with it.for example,the ratings that are given to a movie by a user.
Nominal data : Nominal value is a variable that has no numerical importance,such as occupation,person name etc..
Timeseries data : Time series data has a temporal value attached to it, so this would be something like a date or a time stamp that you can look for trends in time.

In [ ]:
label=LabelEncoder() #decodeするときに、アクセスするために、関数の外で定義
def get_encoded_data(train, test, target):
    
    
    encoded_train=pd.DataFrame() #to make encoded train  
    encoded_test = pd.DataFrame() # to make encoded test  
    label_dict = {} # to make label list at lable encoder 


    for column in  train.columns:
        if column == target:
            encoded_train[column] = label.fit_transform(train[column])
            label_dict[column] = label.classes_
        else:
            if (train[column].dtype=='object') and (test[column].dtype=='object'):
                encoded_train[column]=label.fit_transform(train[column])
                encoded_test[column]=label.fit_transform(test[column])
                label_dict[column] = label.classes_

            else:
                 encoded_train[column]=train[column]
                 encoded_test[column]=test[column]
    return encoded_train, encoded_test, label_dict

In [ ]:
TARGET = 'Transported'
encoded_train, encoded_test, label_dict = get_encoded_data(train = train, test = test, target=TARGET)

In [ ]:
encoded_test

In [ ]:
train

In [ ]:
encoded_train

In [ ]:
encoded_train[TARGET]

In [ ]:
encoded_train.head(3)

In [ ]:
encoded_test.head(3)

In [ ]:
encoded_train.isnull().any()

# k-fold

In [ ]:
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
def get_folds(train_data, target, n_splits=5, shuffle=True, random_state=42):
    
    folds = train_data.copy()
    Fold = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
    for n, (train_index, val_index) in enumerate(Fold.split(folds, folds[target])):
        
        folds.loc[val_index, 'fold'] = int(n)
    folds['fold'] = folds['fold'].astype(int)
    print(folds.groupby(['fold', target]).size())
    return folds

In [ ]:
folds = get_folds(train_data=encoded_train, target='Transported', n_splits=5, shuffle=True, random_state=42)

In [ ]:
folds = encoded_train.copy()
Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds["Transported"])):
    print(val_index)
    folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)
print(folds.groupby(['fold', "Transported"]).size())

In [ ]:
folds

In [ ]:
p_train = folds[folds["fold"] != 0]
p_val = folds[folds["fold"] == 0]

In [ ]:
p_train

In [ ]:
p_train.reset_index(drop=True, inplace=True)
p_val.reset_index(drop=True, inplace=True)

In [ ]:
p_train

In [ ]:
fix_seed(SEED)

In [ ]:
train.columns

In [ ]:
FEATURES = ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Deck', 'Num', 'Side', 'a', 'b', 'c']
TARGET = 'Transported'
p_train[FEATURES]
p_val[TARGET]

In [ ]:
lgb_train = lgb.Dataset(p_train[FEATURES], p_train[TARGET])
lgb_eval = lgb.Dataset(p_val[FEATURES], p_val[TARGET])

In [ ]:
# example of parameters
lgbm_params = {
    'objective': 'binary', # Binary classification : 2値分類ではこれを使う
    'seed': 42, # random seed : これを固定すると、再現性が出る
    'metric': 'auc', 
#    'learning_rate': 0.01,
#    'max_bin': 800, # depth
#    'num_leaves': 80, # leaves,
    "verbose":-1,
    "deterministic":True
}

In [ ]:
#cat_list = ['Embarked']


model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval,
                  verbose_eval=50,  # Learning result output every 50 iterations : 50イテレーション毎に学習結果出力
                  num_boost_round=1000,  # Specify the maximum number of iterations : 最大イテレーション回数指定
                  early_stopping_rounds=100, # Early stopping number : early stoppingを採用するiteration回数
                  #categorical_feature = cat_list # manual categorical feature setting
                 
                 )

In [ ]:
import pickle

model_name = "LGBMmodel.bin"

# saving model
pickle.dump(model, open(model_name, 'wb'))

# loading model
model = pickle.load(open(model_name, 'rb'))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# model.save_model(f'model.txt')
lgb.plot_importance(model, importance_type='gain')
plt.show()

In [ ]:
# predicting validation value
oof_pred = model.predict(p_val[FEATURES], num_iteration=model.best_iteration)

In [ ]:
oof_pred[:3]

In [ ]:
# Calculating AUC (Area Under the Curve)
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(p_val[TARGET], oof_pred)
auc = metrics.auc(fpr, tpr)
print(auc)

# Ploting the ROC curve
plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
plt.legend()
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

In [ ]:
oof_pred

In [ ]:
oof_pred2 = np.where(oof_pred>=0.5,1,0)

In [ ]:
accuracy_score(p_val[TARGET], oof_pred2)

In [ ]:
model = pickle.load(open(model_name, 'rb'))

In [ ]:
encoded_test

In [ ]:
test_X = encoded_test[FEATURES]
test_X

In [ ]:
preds = model.predict(test_X[FEATURES])
preds2 = np.where(preds>=0.5,1,0)
preds2

In [ ]:
label.fit(label_dict[TARGET])
preds3 = label.inverse_transform(preds2)

In [ ]:
len(preds3)

In [ ]:
submission[TARGET] = preds3

In [ ]:
submission[TARGET] = preds3

In [ ]:
submission

In [ ]:
submission.to_csv("submission1.csv",index = False)

# Application:Kfold

## K-fold=1だけまとめる

In [ ]:
fix_seed(SEED)

p_train = folds[folds["fold"] != 0]
p_val = folds[folds["fold"] == 0]

p_train.reset_index(drop=True, inplace=True)
p_val.reset_index(drop=True, inplace=True)


FEATURES = ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Deck', 'Num', 'Side']
TARGET = TARGET

lgb_train = lgb.Dataset(p_train[FEATURES], p_train[TARGET])
lgb_eval = lgb.Dataset(p_val[FEATURES], p_val[TARGET])


model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval,
                  verbose_eval=50,  # Learning result output every 50 iterations : 50イテレーション毎に学習結果出力
                  num_boost_round=1000,  # Specify the maximum number of iterations : 最大イテレーション回数指定
                  early_stopping_rounds=100, # Early stopping number : early stoppingを採用するiteration回数
                  #categorical_feature = cat_list # manual categorical feature setting
                 
                 )



import pickle

model_name = "LGBMmodel.bin"

# saving model
pickle.dump(model, open(model_name, 'wb'))

# loading model
model = pickle.load(open(model_name, 'rb'))

# predicting validation value
oof_pred = model.predict(p_val[FEATURES], num_iteration=model.best_iteration)

oof_pred2 = np.where(oof_pred>=0.5,1,0)

accuracy_score(p_val[TARGET], oof_pred2)

preds = model.predict(test_X[FEATURES])
preds2 = np.where(preds>=0.5,1,0)

# Turn aroud 5 times

In [ ]:
p_val

In [ ]:
def calc_all_k_fold(features: List[str], target: str, lgbm_params: Dict):

    scores = []
    allpreds = []

    allvaliddf = pd.DataFrame()


    for fold in range(5):


        fix_seed(SEED)

        p_train = folds[folds["fold"] != fold]
        p_val = folds[folds["fold"] == fold]

        p_train.reset_index(drop=True, inplace=True)
        p_val.reset_index(drop=True, inplace=True)


        FEATURES = features
        TARGET = target

        lgb_train = lgb.Dataset(p_train[FEATURES], p_train[TARGET])
        lgb_eval = lgb.Dataset(p_val[FEATURES], p_val[TARGET])


        model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval,
                          verbose_eval=50,  # Learning result output every 50 iterations : 50イテレーション毎に学習結果出力
                          num_boost_round=1000,  # Specify the maximum number of iterations : 最大イテレーション回数指定
                          early_stopping_rounds=100, # Early stopping number : early stoppingを採用するiteration回数
                          #categorical_feature = cat_list # manual categorical feature setting

                         )



        import pickle

        model_name = f"LGBMmodel{fold}.bin"

        # saving model
        pickle.dump(model, open(model_name, 'wb'))

        # loading model
        model = pickle.load(open(model_name, 'rb'))

        # predicting validation value
        oof_pred = model.predict(p_val[FEATURES], num_iteration=model.best_iteration)

        oof_pred2 = np.where(oof_pred>=0.5,1,0)

        ##上との変更箇所：
        scores.append(accuracy_score(p_val[TARGET], oof_pred2)) #scoresにaccuracyを代入していく
        #predecting for test_X
        preds = model.predict(test_X[FEATURES])
        #preds2 = np.where(preds>=0.5,1,0)
        allpreds.append(preds)

        # out of fold : oof
        p_val["preds"] = oof_pred2

        allvaliddf = pd.concat([allvaliddf,p_val])
        
        return scores, allpreds, allvaliddf

In [ ]:
calc_all_k_fold(features=FEATURES, target=TARGET, lgbm_params=lgbm_params )

In [ ]:
scores = []
allpreds = []

allvaliddf = pd.DataFrame()


for fold in range(5):


    fix_seed(SEED)

    p_train = folds[folds["fold"] != fold]
    p_val = folds[folds["fold"] == fold]

    p_train.reset_index(drop=True, inplace=True)
    p_val.reset_index(drop=True, inplace=True)


    FEATURES = ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Deck', 'Num', 'Side', 'a', 'b', 'c' ]
    TARGET = TARGET

    lgb_train = lgb.Dataset(p_train[FEATURES], p_train[TARGET])
    lgb_eval = lgb.Dataset(p_val[FEATURES], p_val[TARGET])


    model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval,
                      verbose_eval=50,  # Learning result output every 50 iterations : 50イテレーション毎に学習結果出力
                      num_boost_round=1000,  # Specify the maximum number of iterations : 最大イテレーション回数指定
                      early_stopping_rounds=100, # Early stopping number : early stoppingを採用するiteration回数
                      #categorical_feature = cat_list # manual categorical feature setting

                     )



    import pickle

    model_name = f"LGBMmodel{fold}.bin"

    # saving model
    pickle.dump(model, open(model_name, 'wb'))

    # loading model
    model = pickle.load(open(model_name, 'rb'))

    # predicting validation value
    oof_pred = model.predict(p_val[FEATURES], num_iteration=model.best_iteration)

    oof_pred2 = np.where(oof_pred>=0.5,1,0)
    
    ##上との変更箇所：
    scores.append(accuracy_score(p_val[TARGET], oof_pred2)) #scoresにaccuracyを代入していく
    #predecting for test_X
    preds = model.predict(test_X[FEATURES])
    #preds2 = np.where(preds>=0.5,1,0)
    allpreds.append(preds)
    
    # out of fold : oof
    p_val["preds"] = oof_pred2
    
    allvaliddf = pd.concat([allvaliddf,p_val])

In [ ]:
scores

In [ ]:
allpreds

In [ ]:
p_val

In [ ]:
allpreds

# inference

In [ ]:
allpreds = np.mean(allpreds,axis=0)
allpreds2 = np.where(allpreds>=0.5,1,0)
label.fit(label_dict[TARGET])
allpreds3 = label.inverse_transform(allpreds2)
submission[TARGET] = allpreds3
submission.head(3)

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
def get_subm_binary(allpreds: List[int], target: str):
    
    TARGET = target
    allpreds = np.mean(allpreds,axis=0)
    allpreds2 = np.where(allpreds>=0.5,1,0)
    label.fit(label_dict[TARGET])
    allpreds3 = label.inverse_transform(allpreds2)
    
    return allprds3